# Resonance Overlap Integral – Demo

In [ ]:

# Resonance Overlap Integral – Demo
# This notebook estimates R = ∫ ρ_h(x) ρ_AI(x) dx using Monte Carlo sampling.
import torch
import math
import matplotlib.pyplot as plt

torch.manual_seed(123)

def gauss_logpdf(x, mu, sigma):
    # x: (N, D), mu: (D,), sigma: float
    D = x.shape[1]
    return -0.5 * ((x - mu) ** 2).sum(dim=1) / (sigma ** 2) - 0.5 * D * math.log(2*math.pi*sigma**2)

def rho_gaussian(x, mu, sigma):
    return torch.exp(gauss_logpdf(x, mu, sigma))

def estimate_overlap(n_samples=20000, dim=64, shift=0.5, sigma_h=1.0, sigma_a=1.0):
    x = torch.randn(n_samples, dim)
    mu_h = torch.zeros(dim)
    mu_a = torch.zeros(dim); mu_a[0] = shift  # small shift in 1D for demo
    rh = rho_gaussian(x, mu_h, sigma_h)
    ra = rho_gaussian(x, mu_a, sigma_a)
    R = torch.mean(rh * ra).item()
    return R

dims = [32, 64, 128]
shifts = [0.0, 0.25, 0.5, 1.0]
results = []
for d in dims:
    for s in shifts:
        R = estimate_overlap(n_samples=15000, dim=d, shift=s)
        results.append((d, s, R))

# Display results
for d, s, R in results:
    print(f"dim={d:>3}, shift={s:.2f},  R≈ {R:.6f}")


In [ ]:

# Visualise R vs shift for a fixed dimension
import numpy as np
import matplotlib.pyplot as plt

dim_fixed = 64
shifts = [0.0, 0.1, 0.25, 0.5, 0.75, 1.0]
Rs = []
for s in shifts:
    Rs.append(estimate_overlap(n_samples=20000, dim=dim_fixed, shift=s))

plt.figure(figsize=(6,4))
plt.plot(shifts, Rs, marker="o")
plt.xlabel("Mean shift between ρ_h and ρ_AI (1D of latent)")
plt.ylabel("Estimated overlap R")
plt.title(f"Resonance-Overlap vs Shift (dim={dim_fixed})")
plt.tight_layout()
plt.show()
